In [ ]:
from bloqade import load, save
from bloqade.atom_arrangement import Square
import numpy as np
import os
import matplotlib.pyplot as plt

if not os.path.isdir("data"):
    os.mkdir("data")

# setting the seed
rng = np.random.default_rng(1234)

durations = [0.3, 1.6, 0.3]

mis_udg_program = (
    Square(15, lattice_spacing=5.0)
    .apply_defect_density(0.3, rng=rng)
    .rydberg.rabi.amplitude.uniform.piecewise_linear(durations, [0.0, 15.0, 15.0, 0.0])
    .detuning.uniform.piecewise_linear(
        durations, [-30, -30, "final_detuning", "final_detuning"]
    )
)

mis_udg_job = mis_udg_program.batch_assign(final_detuning=np.linspace(0, 80, 41))

In [ ]:
filename = os.path.join(os.path.abspath(""), "data", "MIS-UDG-job.json")

if not os.path.isfile(filename):
    hw_batch = mis_udg_job.braket.aquila().run_async(shots=100)
    save(hw_batch, filename)

In [ ]:
batch = load(filename)
# batch.fetch()
# save(filename, batch)

In [ ]:
report = batch.report()

average_rydberg_excitation = report.rydberg_densities(filter_perfect_filling=False).sum(
    axis=1
)
final_detunings = report.list_param("final_detuning")

plt.plot(final_detunings, average_rydberg_excitation, color="#6437FF")
plt.xlabel("final detuning (rad/µs)")
plt.ylabel("total rydberg excitations")
plt.show()